In [1]:
# Note: This is a hack to allow importing from the parent directory
import sys
from pathlib import Path

sys.path.append(str(Path().resolve().parent))

# Note: Ignore warnings, be brave (YoLo)
import warnings

warnings.filterwarnings("ignore")

In [2]:
import torch
import optuna
import torch.nn as nn
import torch.optim as optim
from models import ResNetAutoencoder
from data import CIFAR10GaussianSplatsDataset
from utils import train, transform_and_collate

results_path = Path("../logs/resnet_autoencoder_test_2/")
results_path.mkdir(parents=True, exist_ok=True)

# Use one model for the whole splat (separately)
channels_dim = 1
join_mode = "concat"

train_dataset = CIFAR10GaussianSplatsDataset(
    root="../data/CIFAR10GS",
    train=True,
    init_type="grid",
)
val_dataset = CIFAR10GaussianSplatsDataset(
    root="../data/CIFAR10GS",
    val=True,
    init_type="grid",
)

In [ ]:
def objective(trial):
    # Define hyperparameter search space
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    weight_decay = trial.suggest_loguniform("weight_decay", 1e-6, 1e-3)
    loss_fn = trial.suggest_categorical("loss_fn", [nn.L1Loss, nn.MSELoss])
    epochs = trial.suggest_int("epochs", 10, 100, 50)
    grad_clip = trial.suggest_uniform("grad_clip", 0.5, 2.0)
    weight_init = trial.suggest_categorical("weight_init", [True, False])

    # Define train parameters
    model = ResNetAutoencoder(channels_dim=channels_dim, weight_init=weight_init)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=1,
        shuffle=True,
        num_workers=4,
        collate_fn=lambda batch: transform_and_collate(batch, join_mode),
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=1,
        shuffle=False,
        num_workers=4,
        collate_fn=lambda batch: transform_and_collate(batch, join_mode),
    )
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = loss_fn()
    epochs = epochs
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, "min", patience=5)
    grad_clip = grad_clip
    compile_model = True

    results = train(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        criterion=criterion,
        epochs=epochs,
        device=device,
        scheduler=scheduler,
        grad_clip=grad_clip,
        logger=print,
        compile_model=compile_model,
        model_path=results_path / "model.pt",
    )
    return results["val_loss"][-1]


# Run hyperparameter search
study = optuna.create_study(
    direction="minimize", study_name="resnet_autoencoder_test_2"
)
study.optimize(objective, n_trials=100, n_jobs=4)
print(f"Best trial:{study.best_trial}")
with open(results_path / "best_trial.txt", "w") as f:
    f.write(f"Best trial:{study.best_trial}")
optuna.visualization.plot_optimization_history(study).write_image(
    str(results_path / "opt_history.png")
)
optuna.visualization.plot_param_importances(study).write_image(
    str(results_path / "param_importances.png")
)

[I 2025-01-20 18:58:59,645] A new study created in memory with name: resnet_autoencoder_test_2


Epoch 1/1: 100%|██████████| 286/286 [02:58<00:00,  1.60batch/s] 
[I 2025-01-20 19:02:42,947] Trial 0 finished with value: 0.3246388712399442 and parameters: {'lr': 0.1, 'weight_decay': 0.1, 'loss_fn': <class 'torch.nn.modules.loss.L1Loss'>, 'epochs': 1, 'grad_clip': 1.0, 'weight_init': True}. Best is trial 0 with value: 0.3246388712399442.


Epoch 1/1 | Train Loss: 0.3260 | Val Loss: 0.3246
Train Loss: 0.3260 | Val Loss: 0.3246 | Training time: 221.58s
Best trial:FrozenTrial(number=0, state=1, values=[0.3246388712399442], datetime_start=datetime.datetime(2025, 1, 20, 18, 58, 59, 647298), datetime_complete=datetime.datetime(2025, 1, 20, 19, 2, 42, 946783), params={'lr': 0.1, 'weight_decay': 0.1, 'loss_fn': <class 'torch.nn.modules.loss.L1Loss'>, 'epochs': 1, 'grad_clip': 1.0, 'weight_init': True}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.1, log=True, low=0.1, step=None), 'weight_decay': FloatDistribution(high=0.1, log=True, low=0.1, step=None), 'loss_fn': CategoricalDistribution(choices=(<class 'torch.nn.modules.loss.L1Loss'>,)), 'epochs': IntDistribution(high=1, log=False, low=1, step=1), 'grad_clip': FloatDistribution(high=1.0, log=False, low=1.0, step=None), 'weight_init': CategoricalDistribution(choices=(True,))}, trial_id=0, value=None)
